In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import io


from transformers import T5Tokenizer, T5ForConditionalGeneration
import wandb

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.defense_pov = self.data.defense_pov
        self.judgement = self.data.judgement

    def __len__(self):
        return len(self.defense_pov)

    def __getitem__(self, index):
        judgement = str(self.judgement[index])
        judgement = ' '.join(judgement.split())

        defense_pov = str(self.defense_pov[index])
        defense_pov = ' '.join(defense_pov.split())

        source = self.tokenizer.batch_encode_plus([judgement], max_length= self.source_len,truncation=True, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([defense_pov], max_length= self.summ_len,truncation=True, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]

        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)

    return predictions, actuals

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def main():
    wandb.init(project="transformers_tutorials_summarization")
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 1    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 1    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 4       # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 10000
    config.SUMMARY_LEN = 1000

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = AutoTokenizer.from_pretrained("google/long-t5-local-base")



    df = pd.read_csv("train_data.csv")
    df = df[['Perspective-based Summary','Judgement']]
    df.Judgement = 'summarize: ' + df.judgement
    print(df.head())
    train_size = 0.95
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))

    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    torch.cuda.empty_cache()

    model = AutoModelForSeq2SeqLM.from_pretrained("google/long-t5-local-base")
    model = model.to(device)


    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    wandb.watch(model, log="all")

    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        print('Output Files generated for review')

    print(final_df)
    # Save the trained model
    model_path = "T5_Large_local_def"
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)
    print(f"Model and tokenizer saved at: {model_path}")

if __name__ == '__main__':
    main()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


                                         defense_pov  \
0  From the defense attorney's perspective, this ...   
1  From the defense attorney's perspective, this ...   
2  From the perspective of the defense attorney, ...   
3  From the defense attorney's perspective, this ...   
4  From the defense attorney's perspective, this ...   

                                           judgement  
0  summarize: vil Appeal Nos. 1227 to 1230 of 197...  
1  summarize: Appeal No. 2 of 1968.\nAppeal under...  
2  summarize: Civil Appeal Nos. 33763382 of 1988....  
3  summarize: ition No. 2701 of 1981.\n(Under art...  
4  summarize: riminal Appeal No. 3 of 1957.\nAppe...  
FULL Dataset: (3833, 2)
TRAIN Dataset: (3641, 2)
TEST Dataset: (192, 2)


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:667: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Epoch: 0, Loss:  306.6994323730469
Epoch: 0, Loss:  68.10641479492188
Epoch: 0, Loss:  37.87799072265625
Epoch: 0, Loss:  32.39731216430664
Epoch: 0, Loss:  24.24736785888672
Epoch: 0, Loss:  23.255447387695312
Epoch: 0, Loss:  21.900392532348633
Epoch: 0, Loss:  21.662246704101562
Epoch: 0, Loss:  19.41630744934082
Epoch: 0, Loss:  16.870193481445312
Epoch: 0, Loss:  17.751323699951172
Epoch: 0, Loss:  17.544418334960938
Epoch: 0, Loss:  17.838253021240234
Epoch: 0, Loss:  15.480382919311523
Epoch: 0, Loss:  15.651923179626465
Epoch: 0, Loss:  14.738114356994629
Epoch: 0, Loss:  14.300710678100586
Epoch: 0, Loss:  13.004741668701172
Epoch: 0, Loss:  13.22833251953125
Epoch: 0, Loss:  13.062636375427246
Epoch: 0, Loss:  13.843392372131348
Epoch: 0, Loss:  12.560341835021973
Epoch: 0, Loss:  12.073155403137207
Epoch: 0, Loss:  12.169763565063477
Epoch: 0, Loss:  11.445817947387695
Epoch: 0, Loss:  10.357999801635742
Epoch: 0, Loss:  9.651850700378418
Epoch: 0, Loss:  9.802482604980469
E

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2323: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:667: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Completed 100
Output Files generated for review
                                        Generated Text  \
0    The defense could argue that the income tax li...   
1    The defense could argue that the absence of a ...   
2    The defense could argue that the judgment fail...   
3    The defense could argue that the judge's inter...   
4    Movers' case, which could be used as a basis f...   
..                                                 ...   
187  The defense could argue that the Income Tax Ac...   
188  The defense could argue that the subletting of...   
189  The defense could argue that the judgment fail...   
190  The defense could argue that the minimum wages...   
191  The defense could argue that the judgment fail...   

                                           Actual Text  
0    From the perspective of the defense attorney, ...  
1    From the perspective of the defense attorney, ...  
2    From the perspective of the defense attorney, ...  
3    From the perspective o

In [ ]:
!zip -r T5_Large_local_def.zip T5_Large_local_def

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: T5_Large_local_def/ (stored 0%)
  adding: T5_Large_local_def/pytorch_model.bin (deflated 7%)
  adding: T5_Large_local_def/tokenizer.json (deflated 74%)
  adding: T5_Large_local_def/config.json (deflated 49%)
  adding: T5_Large_local_def/tokenizer_config.json (deflated 83%)
  adding: T5_Large_local_def/special_tokens_map.json (deflated 86%)
  adding: T5_Large_local_def/spiece.model (deflated 48%)
